<a href="https://colab.research.google.com/github/abxda/UP_Python_2025/blob/main/Semana_05_01_Miercoles_UP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Celda 1: Instalación de Dependencias
# -------------------------------------
# Se instalan todas las bibliotecas necesarias para el proyecto.
# Es importante reiniciar el entorno de ejecución después de instalar spaCy y su modelo en español
# si se ejecuta por primera vez en una sesión de Colab.

!pip install -U duckduckgo_search -q
!pip install wordcloud matplotlib -q
!pip install transformers -q
!pip install sentence-transformers -q  # Incluye sentence-transformers
!pip install flagembedding -q
!pip install newspaper3k -q
!pip install lxml_html_clean -q # Módulo para limpieza de HTML, aunque no se usa explícitamente en el código provisto

# Instalar spaCy y su modelo en español
!pip install -U spacy -q
!python -m spacy download es_core_news_sm -q

# NLTK (descargas de recursos se harán más adelante cuando se necesiten)

# Para RAG y LLMs locales
!pip install -q ollama chromadb

print("--- Instalaciones completadas ---")
print("NOTA: Si es la primera vez que instalas 'es_core_news_sm' en esta sesión,")
print("      POR FAVOR, REINICIA EL ENTORNO DE EJECUCIÓN (Runtime > Restart runtime)")
print("      antes de continuar con las siguientes celdas.")

In [ ]:
# Celda 2: Importación de Bibliotecas
# -----------------------------------
# Se importan todos los módulos necesarios después de la instalación.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
import spacy
import os
import subprocess
import time
import json

from wordcloud import WordCloud
from duckduckgo_search import DDGS
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN, KMeans
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE
from FlagEmbedding import BGEM3FlagModel
from sentence_transformers import SentenceTransformer

# Para RAG con Ollama y ChromaDB
import ollama
import chromadb

print("--- Bibliotecas importadas ---")

In [ ]:
# Celda 3: Descarga de Recursos NLTK y Carga Modelo spaCy
# -------------------------------------------------------
# Descargamos los recursos necesarios de NLTK (stopwords y tokenizador punkt)
# y cargamos el modelo de spaCy en español.

import nltk # Asegurarse que nltk está importado en esta celda si se ejecuta independientemente
import spacy # Asegurarse que spacy está importado

print("--- Descargando recursos de NLTK (stopwords y punkt) si es necesario ---")
try:
    nltk.download('stopwords', quiet=True) # quiet=True para menos verbosidad si ya está descargado
    nltk.download('punkt', quiet=True)
    print("Recursos NLTK 'stopwords' y 'punkt' verificados/descargados.")
except Exception as e:
    print(f"Error al descargar recursos de NLTK: {e}")
    print("El procesamiento de texto que dependa de estos recursos podría fallar.")

# Cargar el modelo de spaCy en español
# Este paso puede tardar un momento si el modelo es grande o se carga por primera vez.
nlp_spacy = None # Inicializar a None
try:
    nlp_spacy = spacy.load("es_core_news_sm")
    print("--- Modelo spaCy 'es_core_news_sm' cargado correctamente ---")
except OSError:
    print("Error: El modelo 'es_core_news_sm' no se pudo cargar.")
    print("Asegúrate de haberlo descargado (Celda 1) y reiniciado el entorno si era necesario.")
    print("Intenta ejecutar: !python -m spacy download es_core_news_sm")
except Exception as e:
    print(f"Ocurrió un error inesperado al cargar el modelo spaCy: {e}")


# Definir stopwords en español para usar globalmente
# Solo se define si NLTK y el recurso stopwords están disponibles
stop_words_spanish = set()
try:
    from nltk.corpus import stopwords
    stop_words_spanish = set(stopwords.words('spanish'))
    if not stop_words_spanish: # Si por alguna razón devuelve un set vacío pero no hay error
        print("Advertencia: La lista de stopwords en español está vacía. Verifica la descarga de NLTK.")
    else:
        print("Stopwords en español cargadas.")
except LookupError:
    print("Error: Recurso 'stopwords' de NLTK para español no encontrado. La eliminación de stopwords no funcionará.")
except Exception as e:
    print(f"Error al cargar stopwords en español: {e}")

In [ ]:
# Celda 4: Configuración de Búsqueda y Funciones de Obtención de Noticias
# ----------------------------------------------------------------------
# Define los parámetros para la búsqueda de noticias y la función para realizarla.

# Configuraciones para la búsqueda de noticias
# Ejemplo: Buscamos noticias sobre "México aranceles" en español de México.
configuraciones_busqueda = [
    {"idioma": "Español", "keywords": "Mexico Finanzas", "region": "mx-es", "max_results": 100}
]

# Diccionario para almacenar los resultados de noticias por configuración (o idioma si es único)
resultados_noticias_por_config = {}

def buscar_noticias_ddgs(keywords: str, region: str, max_results: int = 25) -> list:
    """
    Busca noticias utilizando la API síncrona de DuckDuckGo Search (DDGS).

    Args:
        keywords (str): Palabras clave para la búsqueda.
        region (str): Código de región (ej. 'mx-es' para México en español).
        max_results (int): Número máximo de resultados a obtener.

    Returns:
        list: Lista de diccionarios, donde cada diccionario representa una noticia.
              Retorna una lista vacía si ocurre un error o no se encuentran noticias.
    """
    print(f"Buscando noticias con keywords='{keywords}', region='{region}', max_results={max_results}...")
    try:
        with DDGS() as ddgs:
            noticias = list(ddgs.news(
                keywords=keywords,
                region=region,
                safesearch='moderate',
                max_results=max_results
            ))
        print(f"Se encontraron {len(noticias)} noticias.")
        return noticias
    except Exception as e:
        print(f"Error al buscar noticias ({keywords}, {region}): {e}")
        return []

def procesar_busquedas_noticias(configs: list) -> pd.DataFrame:
    """
    Itera sobre las configuraciones de búsqueda, obtiene noticias y las consolida en un DataFrame.

    Args:
        configs (list): Lista de diccionarios de configuración de búsqueda.

    Returns:
        pd.DataFrame: DataFrame con todas las noticias encontradas, o DataFrame vacío si no hay resultados.
    """
    todas_las_noticias = []
    for i, conf in enumerate(configs):
        print(f"\nProcesando configuración {i+1}/{len(configs)}: {conf['keywords']}")
        noticias = buscar_noticias_ddgs(
            keywords=conf['keywords'],
            region=conf['region'],
            max_results=conf.get('max_results', 25) # .get para default si no está
        )
        # Añadir información de la configuración a cada noticia (opcional, pero útil)
        for noticia in noticias:
            noticia['query_keywords'] = conf['keywords']
            noticia['query_region'] = conf['region']
        todas_las_noticias.extend(noticias)

    if not todas_las_noticias:
        print("No se encontraron noticias para ninguna configuración.")
        return pd.DataFrame()

    df_noticias_raw = pd.DataFrame(todas_las_noticias)
    print(f"\nTotal de noticias consolidadas: {len(df_noticias_raw)}")
    if not df_noticias_raw.empty:
        print("Primeras noticias encontradas (títulos):")
        for title in df_noticias_raw['title'].head().tolist():
            print(f"- {title}")
    return df_noticias_raw

# --- Ejecución de la búsqueda de noticias ---
df_noticias = procesar_busquedas_noticias(configuraciones_busqueda)

# Mostrar un resumen del DataFrame si no está vacío
if not df_noticias.empty:
    print("\n--- Resumen del DataFrame de Noticias (df_noticias) ---")
    df_noticias.info()
    display(df_noticias.head())
    print(f"Dimensiones del DataFrame: {df_noticias.shape}")
else:
    print("\n--- No se generó el DataFrame de noticias ya que no se encontraron resultados. ---")

In [ ]:
# Celda 4.1 (Corregida): Enriquecimiento de Noticias con Newspaper3k
# -----------------------------------------------------------------
# Esta celda toma el DataFrame 'df_noticias' (creado en Celda 4)
# e intenta descargar el contenido completo de cada noticia usando su URL
# con la biblioteca newspaper3k. El contenido se usará para reemplazar
# o complementar la columna 'body'.

from newspaper import Article, Config # Asegurarse de la importación
import pandas as pd # Para trabajar con el DataFrame
import time # Para añadir pequeños delays y ser cortés con los servidores

print("\n--- Iniciando Enriquecimiento de Noticias con Newspaper3k ---")

# Verificar si df_noticias existe y tiene la columna 'url'
if 'df_noticias' in globals() and not globals()['df_noticias'].empty and 'url' in globals()['df_noticias'].columns:
    df_to_enrich = globals()['df_noticias']

    df_to_enrich['full_text_newspaper'] = None
    df_to_enrich['newspaper_download_error'] = None

    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    config.request_timeout = 15
    config.fetch_images = False
    config.memoize_articles = True
    config.verbose = False

    print(f"Procesando {len(df_to_enrich)} URLs para extraer texto completo...")

    for index, row in df_to_enrich.iterrows():
        url = row['url']
        print(f"\nProcesando URL ({index + 1}/{len(df_to_enrich)}): {url}")

        if not url or not isinstance(url, str) or not url.startswith(('http://', 'https://')):
            print(f"  URL inválida o vacía: '{url}'. Omitiendo.")
            df_to_enrich.loc[index, 'newspaper_download_error'] = "URL inválida o vacía"
            continue

        try:
            article = Article(url, config=config)

            print("  Descargando HTML...")
            article.download()

            print("  Parseando artículo...")
            article.parse()

            extracted_text = article.text

            if extracted_text and len(extracted_text) > 50:
                preview_text = extracted_text[:100].replace('\n', ' ')
                print(f"  Texto extraído (primeros 100 chars): {preview_text}...")
                print(f"  Longitud del texto extraído: {len(extracted_text)}")
                df_to_enrich.loc[index, 'full_text_newspaper'] = extracted_text
            else:
                print("  No se pudo extraer texto significativo o el texto es muy corto.")
                df_to_enrich.loc[index, 'newspaper_download_error'] = "Texto extraído vacío o corto"

        except Exception as e:
            print(f"  Error al procesar URL '{url}': {e}")
            df_to_enrich.loc[index, 'newspaper_download_error'] = str(e)

        time.sleep(0.5)

    print("\n--- Enriquecimiento completado. Verificando resultados... ---")

    df_to_enrich['body_enriquecido'] = df_to_enrich['body']

    for index, row in df_to_enrich.iterrows():
        texto_newspaper = row['full_text_newspaper']
        texto_body_original = row['body']

        if pd.notna(texto_newspaper) and len(texto_newspaper) > (len(str(texto_body_original)) if pd.notna(texto_body_original) else 0):
            df_to_enrich.loc[index, 'body_enriquecido'] = texto_newspaper

    actualizados = (df_to_enrich['body_enriquecido'] != df_to_enrich['body']).sum()
    print(f"\nSe actualizaron/mejoraron los cuerpos de {actualizados} noticias (de un total de {len(df_to_enrich)}).")

    print("\nEjemplo de 'body', 'full_text_newspaper' y 'body_enriquecido':")
    display(df_to_enrich[['url', 'title', 'body', 'full_text_newspaper', 'body_enriquecido', 'newspaper_download_error']].head(10))

    globals()['df_noticias'] = df_to_enrich
    print("\nDataFrame 'df_noticias' actualizado con contenido enriquecido.")

else:
    if 'df_noticias' not in globals() or globals()['df_noticias'].empty:
        print("El DataFrame 'df_noticias' no existe o está vacío. No se puede enriquecer.")
    elif 'url' not in globals()['df_noticias'].columns:
        print("La columna 'url' no existe en 'df_noticias'. No se puede enriquecer.")

print("\n--- Fin del Proceso de Enriquecimiento de Noticias ---")

In [ ]:
# Celda 5: Función para Generar Nube de Palabras y Visualización Inicial
# ---------------------------------------------------------------------
# Función reutilizable para generar nubes de palabras y una visualización
# inicial si hay noticias.

def generar_nube_de_palabras_wc(texto_completo: str, titulo_grafico: str = "Nube de Palabras"):
    """
    Genera y muestra una nube de palabras a partir de un texto.

    Args:
        texto_completo (str): El string concatenado de todos los textos a visualizar.
        titulo_grafico (str): Título para el gráfico de la nube de palabras.
    """
    if not texto_completo.strip():
        print(f"No hay suficiente texto para generar la nube de palabras: '{titulo_grafico}'.")
        return

    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto_completo)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(titulo_grafico)
    plt.show()

# Generar nube de palabras del texto original de las noticias (columna 'body')
if not df_noticias.empty and 'body_enriquecido' in df_noticias.columns:
    # Asegurarse de que los cuerpos de las noticias sean strings y manejar NaNs
    textos_originales = df_noticias['body_enriquecido'].fillna('').astype(str)
    news_text_original = ' '.join(textos_originales)
    generar_nube_de_palabras_wc(news_text_original, "Nube de Palabras (Texto Original 'body')")
else:
    print("No se puede generar la nube de palabras inicial: DataFrame vacío o sin columna 'body'.")

In [ ]:
# Celda 6: Limpieza de Texto y Lemmatización
# ------------------------------------------
# Funciones para limpiar texto (minúsculas, puntuación, stopwords) y lematizar.
# Se aplican estas transformaciones al DataFrame.

def limpiar_texto(texto: str) -> str:
    """
    Limpia un texto: convierte a minúsculas, elimina puntuación y stopwords.

    Args:
        texto (str): Texto a limpiar.

    Returns:
        str: Texto limpio.
    """
    if not isinstance(texto, str): # Manejar posibles NaNs o no-strings
        return ""
    texto_lower = texto.lower()
    texto_sin_puntuacion = texto_lower.translate(str.maketrans('', '', string.punctuation))
    palabras = texto_sin_puntuacion.split()
    palabras_filtradas = [palabra for palabra in palabras if palabra not in stop_words_spanish]
    return ' '.join(palabras_filtradas)

def lematizar_texto(texto: str) -> str:
    """
    Lematiza un texto utilizando spaCy.

    Args:
        texto (str): Texto a lematizar (generalmente ya limpio de stopwords y puntuación).

    Returns:
        str: Texto lematizado.
    """
    if not isinstance(texto, str) or not nlp_spacy: # Manejar no-strings o si spaCy no cargó
        return ""
    doc = nlp_spacy(texto)
    return ' '.join([token.lemma_ for token in doc if token.lemma_.strip()]) # Evitar lemas vacíos

if not df_noticias.empty and 'body' in df_noticias.columns:
    print("\n--- Limpiando y Lematizando Textos ---")
    # 1. Crear columna 'clean_body'
    df_noticias['clean_body'] = df_noticias['body'].fillna('').astype(str).apply(limpiar_texto)
    print("Columna 'clean_body' creada.")

    # 2. Crear columna 'lemmatized_body' a partir de 'clean_body'
    df_noticias['lemmatized_body'] = df_noticias['clean_body'].apply(lematizar_texto)
    print("Columna 'lemmatized_body' creada.")

    display(df_noticias[['title', 'body', 'clean_body', 'lemmatized_body']].head())

    # Generar nube de palabras con texto limpio
    textos_limpios_concat = ' '.join(df_noticias['clean_body'])
    generar_nube_de_palabras_wc(textos_limpios_concat, "Nube de Palabras (Texto Limpio 'clean_body')")

    # Generar nube de palabras con texto lematizado
    textos_lematizados_concat = ' '.join(df_noticias['lemmatized_body'])
    generar_nube_de_palabras_wc(textos_lematizados_concat, "Nube de Palabras (Texto Lematizado 'lemmatized_body')")
else:
    print("No se puede procesar texto: DataFrame vacío o sin columna 'body'.")

In [ ]:
# Celda 7: Análisis de Fuentes de Noticias
# ----------------------------------------
# Contar y graficar la cantidad de noticias por fuente.

if not df_noticias.empty and 'source' in df_noticias.columns:
    print("\n--- Análisis de Fuentes de Noticias ---")
    conteo_fuentes = df_noticias['source'].value_counts()

    plt.figure(figsize=(12, max(8, len(conteo_fuentes) * 0.5))) # Ajustar altura dinámicamente
    conteo_fuentes.plot(kind='barh', color='skyblue') # 'barh' para mejor lectura de muchas fuentes
    plt.title('Cantidad de Noticias por Fuente')
    plt.xlabel('Cantidad de Noticias')
    plt.ylabel('Fuente')
    plt.gca().invert_yaxis() # La fuente más común arriba
    plt.tight_layout()
    plt.show()

    print("\nConteo de noticias por fuente:")
    print(conteo_fuentes)
else:
    print("No se puede analizar fuentes: DataFrame vacío o sin columna 'source'.")

In [ ]:
# Celda 8: Funciones para Clustering y Visualización de Clusters
# -------------------------------------------------------------
# Define funciones para preprocesar texto específicamente para algunos modelos (si es necesario),
# y una función robusta para visualizar los clusters con nubes de palabras y títulos.

def preprocess_text_for_tfidf_like_models(text: str) -> str:
    """
    Preprocesa texto para modelos como TF-IDF: lematiza y elimina stopwords/puntuación.
    Es similar a la combinación de limpiar_texto y lematizar_texto pero en un solo paso.
    """
    if not isinstance(text, str) or not nlp_spacy:
        return ""
    doc = nlp_spacy(text.lower()) # Asegurar minúsculas antes de procesar con spaCy
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.lemma_.strip()])

def generar_nube_palabras_para_cluster(textos_cluster: list, ax):
    """
    Genera una nube de palabras para un conjunto de textos y la dibuja en un subplot (ax).
    """
    if not textos_cluster:
        ax.text(0.5, 0.5, "No hay texto para esta nube", ha='center', va='center')
        ax.axis('off')
        return

    texto_completo = ' '.join(textos_cluster)
    if not texto_completo.strip():
        ax.text(0.5, 0.5, "Texto vacío para esta nube", ha='center', va='center')
        ax.axis('off')
        return

    wordcloud = WordCloud(width=400, height=200, background_color='white', max_words=50).generate(texto_completo)
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis('off')

def visualizar_clusters_detalle(df_original: pd.DataFrame, clusters_labels: np.ndarray, texto_col_for_wc: str, titulo_general: str):
    """
    Visualiza clusters mostrando una nube de palabras y los títulos de las noticias para cada cluster.

    Args:
        df_original (pd.DataFrame): DataFrame original que contiene 'title', 'source', y la columna de texto para la WC.
        clusters_labels (np.ndarray): Array con las etiquetas de cluster para cada noticia.
        texto_col_for_wc (str): Nombre de la columna en df_original a usar para generar las nubes de palabras (ej: 'lemmatized_body').
        titulo_general (str): Título general para la visualización.
    """
    df_copy = df_original.copy()
    df_copy['cluster'] = clusters_labels

    # Contar noticias por cluster, excluyendo el cluster -1 (ruido en DBSCAN)
    cluster_counts = df_copy[df_copy['cluster'] != -1]['cluster'].value_counts()
    num_clusters_validos = len(cluster_counts)

    if num_clusters_validos == 0:
        print(f"{titulo_general}: No se formaron clusters válidos (todos podrían ser ruido -1).")
        print(f"Distribución de clusters (incluye ruido): \n{pd.Series(clusters_labels).value_counts()}")
        return

    print(f"\n--- {titulo_general} ---")
    print(f"Distribución de clusters: \n{pd.Series(clusters_labels).value_counts().sort_index()}")

    # Crear una figura con subplots: una fila por cluster, dos columnas (WC, Títulos)
    # Ajustar el tamaño de la figura dinámicamente
    fig_height = max(10, num_clusters_validos * 4) # 4 pulgadas de alto por cluster
    fig, axs = plt.subplots(num_clusters_validos, 2, figsize=(20, fig_height), squeeze=False) # squeeze=False para asegurar axs 2D

    valid_cluster_indices = cluster_counts.index.sort_values()

    for i, cluster_id in enumerate(valid_cluster_indices):
        cluster_data = df_copy[df_copy['cluster'] == cluster_id]
        textos_cluster = cluster_data[texto_col_for_wc].tolist()

        # Limitar el número de títulos a mostrar para no saturar
        titulos_cluster = [f"{idx}. {row['title']} ({row['source']})" for idx, (_, row) in enumerate(cluster_data[['title', 'source']].head(10).iterrows(), 1)]

        # Nube de palabras en la primera columna del subplot
        generar_nube_palabras_para_cluster(textos_cluster, axs[i, 0])
        axs[i, 0].set_title(f'Cluster {cluster_id} - Nube ({len(textos_cluster)} noticias)')

        # Títulos en la segunda columna del subplot
        if titulos_cluster:
            axs[i, 1].text(0.01, 0.99, "\n".join(titulos_cluster), verticalalignment='top', horizontalalignment='left', fontsize=9, family='monospace', wrap=True)
        else:
            axs[i, 1].text(0.5, 0.5, "Sin títulos para mostrar.", ha='center', va='center')
        axs[i, 1].axis('off')
        axs[i, 1].set_title(f'Cluster {cluster_id} - Títulos y Fuentes (primeros 10)')

    plt.suptitle(titulo_general, fontsize=16, y=1.01) # y>1 para que no solape con subplots
    plt.tight_layout(pad=2.0)
    plt.show()

# Preprocesar texto para TF-IDF si el DataFrame existe y tiene 'body'
if not df_noticias.empty and 'body_enriquecido' in df_noticias.columns:
    df_noticias['processed_for_tfidf'] = df_noticias['body_enriquecido'].fillna('').astype(str).apply(preprocess_text_for_tfidf_like_models)
    print("Columna 'processed_for_tfidf' creada para TF-IDF.")
else:
    print("No se puede crear 'processed_for_tfidf': DataFrame vacío o sin columna 'body_enriquecido'.")

In [ ]:
# Celda 9: Clustering con TF-IDF
# -------------------------------
# Vectorización con TF-IDF (usando n-gramas de caracteres) y aplicación de DBSCAN y KMeans.

if not df_noticias.empty and 'lemmatized_body' in df_noticias.columns and not df_noticias['lemmatized_body'].str.strip().eq('').all():
    print("\n--- Clustering con TF-IDF (sobre 'lemmatized_body') ---")

    # Usaremos 'lemmatized_body' que ya está bastante procesado.
    # Para TF-IDF con n-gramas de caracteres, el texto original o mínimamente procesado puede ser mejor.
    # Aquí se usa 'lemmatized_body' como en el original, pero considera 'clean_body' o 'body' si los resultados no son buenos.
    textos_para_tfidf = df_noticias['lemmatized_body'].tolist()

    if not any(textos_para_tfidf): # Chequea si todos los textos están vacíos
        print("Los textos para TF-IDF están vacíos. Saltando clustering TF-IDF.")
    else:
        # Vectorización con TF-IDF (n-gramas de caracteres)
        vectorizer_tfidf_char = TfidfVectorizer(analyzer='char', ngram_range=(3, 5), min_df=2) # min_df=2 para ignorar términos muy raros
        X_tfidf_char = vectorizer_tfidf_char.fit_transform(textos_para_tfidf)
        X_tfidf_char_normalized = normalize(X_tfidf_char)
        print(f"Dimensiones de la matriz TF-IDF (char n-grams): {X_tfidf_char_normalized.shape}")

        if X_tfidf_char_normalized.shape[0] > 0 and X_tfidf_char_normalized.shape[1] > 0: # Asegurarse que la matriz no está vacía
            # DBSCAN con TF-IDF
            # Ajustar 'eps' y 'min_samples' según la densidad de tus datos
            dbscan_tfidf = DBSCAN(eps=0.8, min_samples=2, metric='cosine')
            clusters_dbscan_tfidf = dbscan_tfidf.fit_predict(X_tfidf_char_normalized)
            visualizar_clusters_detalle(df_noticias, clusters_dbscan_tfidf, 'lemmatized_body', "Clusters TF-IDF + DBSCAN (char n-grams)")

            # KMeans con TF-IDF
            # Definir el número de clusters (k). Podría estimarse con Elbow method, Silhouette score, etc.
            num_k_clusters_tfidf = min(8, X_tfidf_char_normalized.shape[0]) # No más clusters que muestras
            if num_k_clusters_tfidf > 1:
                kmeans_tfidf = KMeans(n_clusters=num_k_clusters_tfidf, random_state=42, n_init='auto')
                clusters_kmeans_tfidf = kmeans_tfidf.fit_predict(X_tfidf_char_normalized)
                visualizar_clusters_detalle(df_noticias, clusters_kmeans_tfidf, 'lemmatized_body', f"Clusters TF-IDF + KMeans (k={num_k_clusters_tfidf}, char n-grams)")
            else:
                print("No hay suficientes muestras para KMeans con TF-IDF (se necesitan al menos 2).")
        else:
            print("La matriz TF-IDF está vacía o tiene dimensiones cero. Saltando clustering TF-IDF.")
else:
    print("No se puede realizar clustering TF-IDF: DataFrame vacío, sin 'lemmatized_body' o columna vacía.")

In [ ]:
# Celda 10: Preparación de Embeddings BGE-M3
# -----------------------------------------
# Carga del modelo BGE-M3 y generación de embeddings para los textos limpios.

# Cargar el modelo BGE-M3
try:
    model_bge_m3 = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True) # use_fp16 para acelerar si GPU disponible
    print("--- Modelo BGE-M3 ('BAAI/bge-m3') cargado ---")
except Exception as e:
    print(f"Error al cargar el modelo BGE-M3: {e}")
    model_bge_m3 = None

def generar_embeddings_bge_m3(textos: list, model) -> np.ndarray:
    """Genera embeddings BGE-M3 para una lista de textos."""
    if not model or not textos:
        return np.array([])
    print(f"Generando embeddings BGE-M3 para {len(textos)} textos...")
    # BGE-M3 no requiere prefijos como "passage:" o "query:" para la codificación directa
    embeddings_output = model.encode(textos, return_dense=True, return_sparse=False, return_colbert_vecs=False)
    return embeddings_output['dense_vecs']


embeddings_bge_m3_list = []
if model_bge_m3 and not df_noticias.empty and 'clean_body' in df_noticias.columns and not df_noticias['clean_body'].str.strip().eq('').all():
    textos_para_bge_m3 = df_noticias['clean_body'].tolist()
    if not any(textos_para_bge_m3):
        print("Los textos para BGE-M3 están vacíos. Saltando generación de embeddings BGE-M3.")
    else:
        embeddings_bge_m3_list = generar_embeddings_bge_m3(textos_para_bge_m3, model_bge_m3)
        if embeddings_bge_m3_list.size > 0:
            X_bge_m3_normalized = normalize(embeddings_bge_m3_list, norm='l2', axis=1) # Normalizar es buena práctica para distancias coseno
            print(f"Embeddings BGE-M3 generados y normalizados. Dimensiones: {X_bge_m3_normalized.shape}")
        else:
            print("No se generaron embeddings BGE-M3.")
            X_bge_m3_normalized = np.array([]) # Asegurar que es un array vacío
else:
    print("No se pueden generar embeddings BGE-M3: modelo no cargado, DataFrame vacío, sin 'clean_body' o columna vacía.")
    X_bge_m3_normalized = np.array([])

In [ ]:
# Celda 11: Clustering con Embeddings BGE-M3
# ------------------------------------------
# Aplicación de KMeans y DBSCAN a los embeddings BGE-M3.

if 'X_bge_m3_normalized' in locals() and X_bge_m3_normalized.size > 0:
    print("\n--- Clustering con Embeddings BGE-M3 ---")

    # KMeans con BGE-M3
    num_k_clusters_bge = min(8, X_bge_m3_normalized.shape[0]) # No más clusters que muestras
    if num_k_clusters_bge > 1:
        kmeans_bge = KMeans(n_clusters=num_k_clusters_bge, random_state=42, n_init='auto')
        clusters_kmeans_bge = kmeans_bge.fit_predict(X_bge_m3_normalized)
        visualizar_clusters_detalle(df_noticias, clusters_kmeans_bge, 'clean_body', f"Clusters BGE-M3 + KMeans (k={num_k_clusters_bge})")
    else:
        print("No hay suficientes muestras para KMeans con BGE-M3 (se necesitan al menos 2).")

    # DBSCAN con BGE-M3
    # 'eps' para DBSCAN es sensible. Puede requerir ajuste.
    # Para embeddings normalizados, 'cosine' o 'euclidean' pueden funcionar.
    # Si se usa 'euclidean' con L2-normalized vectors, la distancia euclidiana está relacionada con la similaridad coseno.
    # d_euc(A,B)^2 = 2 - 2 * cos_sim(A,B)
    # Un 'eps' típico para embeddings normalizados con 'euclidean' podría estar entre 0.5 y 1.2
    dbscan_bge = DBSCAN(eps=0.85, min_samples=2, metric='euclidean') # Usar 'cosine' y ajustar eps si es preferible.
    clusters_dbscan_bge = dbscan_bge.fit_predict(X_bge_m3_normalized)
    visualizar_clusters_detalle(df_noticias, clusters_dbscan_bge, 'clean_body', "Clusters BGE-M3 + DBSCAN")
else:
    print("No se pueden realizar clustering con BGE-M3: embeddings no generados.")

In [ ]:
# Celda 12: Visualización t-SNE de Embeddings BGE-M3
# -------------------------------------------------
# Reducción de dimensionalidad con t-SNE y visualización de los embeddings BGE-M3.

if 'X_bge_m3_normalized' in locals() and X_bge_m3_normalized.size > 0 and 'title' in df_noticias.columns:
    print("\n--- Visualización t-SNE de Embeddings BGE-M3 ---")
    num_samples_bge = X_bge_m3_normalized.shape[0]
    perplexity_bge = min(30, max(1, num_samples_bge - 1)) # Ajustar perplexity si hay pocos datos

    if num_samples_bge > 1 : # t-SNE necesita más de 1 muestra
        tsne_bge = TSNE(n_components=2, random_state=42, perplexity=perplexity_bge, n_iter=1000, metric='cosine')
        X_bge_2d = tsne_bge.fit_transform(X_bge_m3_normalized)

        df_tsne_bge = pd.DataFrame(X_bge_2d, columns=['x', 'y'])
        # Asegurarse que el índice coincida si df_noticias fue filtrado o tiene saltos
        df_tsne_bge.index = df_noticias.index[:len(df_tsne_bge)] # Alinear con las primeras N filas de df_noticias
        df_tsne_bge['titulo'] = df_noticias['title']

        plt.figure(figsize=(18, 12))
        sns.scatterplot(data=df_tsne_bge, x='x', y='y', hue='titulo', palette='viridis', legend=False, alpha=0.7)
        for i in df_tsne_bge.index:
            plt.text(df_tsne_bge.loc[i, 'x'], df_tsne_bge.loc[i, 'y'], df_tsne_bge.loc[i, 'titulo'],
                     fontsize=8, color='black', alpha=0.7,
                     bbox=dict(facecolor='white', alpha=0.3, edgecolor='none', pad=0.1))
        plt.title('t-SNE de Embeddings de Noticias (BGE-M3)')
        plt.xlabel('Componente t-SNE 1')
        plt.ylabel('Componente t-SNE 2')
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.show()
    else:
        print("No hay suficientes muestras para la visualización t-SNE de BGE-M3 (se necesita >1).")
else:
    print("No se puede realizar t-SNE de BGE-M3: embeddings no disponibles o DataFrame sin 'title'.")

In [ ]:
# Celda 13 (Modificada): Carga del Modelo de Embedding Sentence-Transformer
# -----------------------------------------------------------------------
# Carga del modelo 'paraphrase-multilingual-MiniLM-L12-v2' para RAG.

from sentence_transformers import SentenceTransformer # Asegurar importación
import numpy as np # Para manipulación de arrays
from sklearn.preprocessing import normalize # Para normalizar embeddings

print("\n--- Cargando Modelo de Embedding para RAG ---")

# Nombre del modelo Sentence-Transformer a utilizar
embedding_model_name_for_rag = 'paraphrase-multilingual-MiniLM-L12-v2'
active_st_model_instance = None # Variable para el modelo cargado
X_st_embeddings_normalized = np.array([]) # Para los embeddings de documentos

print(f"Intentando cargar el modelo Sentence-Transformer: '{embedding_model_name_for_rag}'...")
try:
    active_st_model_instance = SentenceTransformer(embedding_model_name_for_rag)
    print(f"Modelo '{embedding_model_name_for_rag}' cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar '{embedding_model_name_for_rag}': {e}.")
    print("Asegúrate de tener conexión a internet si el modelo no está cacheado.")
    active_st_model_instance = None

# Si el modelo se cargó y tenemos datos, generamos los embeddings para los documentos
if active_st_model_instance and 'df_noticias' in globals() and not globals()['df_noticias'].empty and \
   'clean_body' in globals()['df_noticias'].columns and \
   not globals()['df_noticias']['clean_body'].str.strip().eq('').all():

    print(f"\nGenerando embeddings con '{embedding_model_name_for_rag}' para los documentos...")
    textos_documentos = globals()['df_noticias']['clean_body'].tolist()

    # Estos modelos generalmente no requieren prefijos como "passage:"
    # Pero si se usa un modelo tipo E5, aquí se añadiría el prefijo.
    # Para MiniLM, no se necesita.

    if not any(textos_documentos): # Chequea si todos los textos están vacíos
        print("Los textos para generar embeddings están vacíos.")
    else:
        try:
            # Generar embeddings
            raw_embeddings = active_st_model_instance.encode(textos_documentos, show_progress_bar=True)

            # Normalizar los embeddings (buena práctica para distancia coseno)
            X_st_embeddings_normalized = normalize(raw_embeddings, norm='l2', axis=1)
            print(f"Embeddings generados y normalizados con '{embedding_model_name_for_rag}'. Dimensiones: {X_st_embeddings_normalized.shape}")
        except Exception as e_embed_docs:
            print(f"Error al generar embeddings para documentos con '{embedding_model_name_for_rag}': {e_embed_docs}")
            X_st_embeddings_normalized = np.array([]) # Asegurar que es un array vacío si falla
else:
    if not active_st_model_instance:
        print(f"El modelo '{embedding_model_name_for_rag}' no se pudo cargar. No se generarán embeddings de documentos.")
    else:
        print("No se pueden generar embeddings para documentos: DataFrame 'df_noticias' no está listo o 'clean_body' está vacío.")

# Esta celda ahora define 'active_st_model_instance' y 'X_st_embeddings_normalized'
# que serán usados por la Celda 17.

In [ ]:
# Celda 14 (Adaptada): Clustering con Embeddings MiniLM ('paraphrase-multilingual-MiniLM-L12-v2')
# -----------------------------------------------------------------------------------------
# Aplicación de KMeans y DBSCAN a los embeddings MiniLM generados en Celda 13.

# Necesitarás importar KMeans, DBSCAN, y la función visualizar_clusters_detalle si no están ya en el scope.
# from sklearn.cluster import KMeans, DBSCAN
# from sklearn.preprocessing import normalize # Ya debería estar, pero por si acaso
# (asumimos que visualizar_clusters_detalle ya está definida en una celda anterior, como Celda 8)

if 'X_st_embeddings_normalized' in globals() and globals()['X_st_embeddings_normalized'].size > 0 and \
   'df_noticias' in globals() and not globals()['df_noticias'].empty:

    print("\n--- Clustering con Embeddings MiniLM ('paraphrase-multilingual-MiniLM-L12-v2') ---")

    current_embeddings_for_clustering = globals()['X_st_embeddings_normalized']
    df_for_visualization = globals()['df_noticias']

    # KMeans con MiniLM
    # Ajustar num_k_clusters según el número de muestras y la estructura esperada.
    num_k_clusters_minilm = min(8, current_embeddings_for_clustering.shape[0])
    if num_k_clusters_minilm > 1:
        print(f"\nEjecutando KMeans con k={num_k_clusters_minilm}...")
        kmeans_minilm = KMeans(n_clusters=num_k_clusters_minilm, random_state=42, n_init='auto')
        clusters_kmeans_minilm = kmeans_minilm.fit_predict(current_embeddings_for_clustering)
        visualizar_clusters_detalle(df_for_visualization,
                                    clusters_kmeans_minilm,
                                    'clean_body', # Columna de texto para nubes de palabras
                                    f"Clusters MiniLM + KMeans (k={num_k_clusters_minilm})")
    else:
        print("No hay suficientes muestras para KMeans con MiniLM (se necesitan al menos 2).")

    # DBSCAN con MiniLM
    # El valor de 'eps' para DBSCAN es muy sensible y depende de la escala/densidad de tus embeddings.
    # Para embeddings L2 normalizados, la distancia euclidiana está relacionada con la similaridad coseno.
    # eps_euc ~ sqrt(2 - 2*cos_sim_threshold). Si quieres cos_sim > 0.7, eps ~ sqrt(2-1.4) = sqrt(0.6) ~ 0.77
    # Necesitarás experimentar con 'eps' y 'min_samples'.
    eps_dbscan_minilm = 0.75 # VALOR DE EJEMPLO, AJUSTAR
    min_samples_dbscan_minilm = 2
    print(f"\nEjecutando DBSCAN con eps={eps_dbscan_minilm}, min_samples={min_samples_dbscan_minilm}...")
    dbscan_minilm = DBSCAN(eps=eps_dbscan_minilm, min_samples=min_samples_dbscan_minilm, metric='euclidean') # 'cosine' también es una opción
    clusters_dbscan_minilm = dbscan_minilm.fit_predict(current_embeddings_for_clustering)
    visualizar_clusters_detalle(df_for_visualization,
                                clusters_dbscan_minilm,
                                'clean_body',
                                f"Clusters MiniLM + DBSCAN (eps={eps_dbscan_minilm})")
else:
    print("No se pueden realizar clustering con MiniLM: embeddings no generados o DataFrame no disponible.")

In [ ]:
# Celda 15 (Adaptada): Visualización t-SNE de Embeddings MiniLM ('paraphrase-multilingual-MiniLM-L12-v2')
# ----------------------------------------------------------------------------------------------------
# Reducción de dimensionalidad con t-SNE y visualización de los embeddings MiniLM.

# Necesitarás importar TSNE, matplotlib.pyplot, seaborn si no están ya en el scope.
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pandas as pd

if 'X_st_embeddings_normalized' in globals() and globals()['X_st_embeddings_normalized'].size > 0 and \
   'df_noticias' in globals() and 'title' in globals()['df_noticias'].columns:

    print("\n--- Visualización t-SNE de Embeddings MiniLM ('paraphrase-multilingual-MiniLM-L12-v2') ---")

    current_embeddings_for_tsne = globals()['X_st_embeddings_normalized']
    df_for_labels = globals()['df_noticias']

    num_samples_minilm = current_embeddings_for_tsne.shape[0]
    # Perplexity debe ser menor que el número de muestras. Comúnmente entre 5 y 50.
    perplexity_minilm = min(30, max(1, num_samples_minilm - 1))

    if num_samples_minilm > 1: # t-SNE necesita más de 1 muestra
        print(f"Ejecutando t-SNE con perplexity={perplexity_minilm}...")
        tsne_minilm = TSNE(n_components=2,
                           random_state=42,
                           perplexity=perplexity_minilm,
                           n_iter=1000,
                           metric='cosine') # 'cosine' es bueno para embeddings de texto normalizados
        X_minilm_2d = tsne_minilm.fit_transform(current_embeddings_for_tsne)

        df_tsne_minilm = pd.DataFrame(X_minilm_2d, columns=['x', 'y'])
        # Asegurar que el índice coincida si df_noticias fue filtrado o tiene saltos
        # Tomar solo los títulos correspondientes a los embeddings procesados
        df_tsne_minilm.index = df_for_labels.index[:len(df_tsne_minilm)]
        df_tsne_minilm['titulo'] = df_for_labels['title']

        plt.figure(figsize=(18, 12))
        sns.scatterplot(data=df_tsne_minilm, x='x', y='y', hue='titulo', palette='viridis', legend=False, alpha=0.7)

        # Ajustar el número de etiquetas para evitar sobrecargar el gráfico si hay muchos puntos
        num_labels_to_show = min(len(df_tsne_minilm), 75) # Mostrar hasta 75 etiquetas
        indices_to_label = df_tsne_minilm.sample(n=num_labels_to_show, random_state=42).index if len(df_tsne_minilm) > num_labels_to_show else df_tsne_minilm.index

        for i in indices_to_label:
            if i in df_tsne_minilm.index: # Doble chequeo
                plt.text(df_tsne_minilm.loc[i, 'x'], df_tsne_minilm.loc[i, 'y'], df_tsne_minilm.loc[i, 'titulo'],
                         fontsize=8, color='black', alpha=0.7,
                         bbox=dict(facecolor='white', alpha=0.3, edgecolor='none', pad=0.1))

        plt.title("t-SNE de Embeddings de Noticias (paraphrase-multilingual-MiniLM-L12-v2)")
        plt.xlabel('Componente t-SNE 1')
        plt.ylabel('Componente t-SNE 2')
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.show()
    else:
        print("No hay suficientes muestras para la visualización t-SNE de MiniLM (se necesita >1).")
else:
    print("No se puede realizar t-SNE de MiniLM: embeddings no disponibles, DataFrame sin 'title' o vacío.")

In [ ]:
#!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
#!sudo apt-get update && sudo apt-get install -y cuda-drivers

In [ ]:
#import os
#os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

In [ ]:
#!nvidia-smi

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
#SI MARCO ERRORES ES QUE NO ESTAS USANDO UN ENTORNO GPU T4

In [ ]:
# Celda 16 (Ajustada): Inicio de Ollama Server y Descarga de Modelos LLM
# -------------------------------------------------------------------------
# Asumimos que Ollama ya fue instalado en una celda anterior.
# Esta celda se enfoca en iniciar el servidor Ollama y descargar los modelos LLM necesarios.

import subprocess
import time
import ollama # Asegurarse que ollama está importado

print("--- Configurando e iniciando Ollama Server (asumiendo Ollama ya instalado) ---")
ollama_process = None # Inicializar la variable

# Intentar iniciar el servidor Ollama si no está corriendo
try:
    # Verificar si ya hay un proceso ollama sirviendo
    # Usamos 'pgrep ollama' que es más general que 'pgrep -f ollama serve'
    # ya que el proceso principal de Ollama podría no incluir 'serve' en su nombre exacto en todas las plataformas
    result = subprocess.run(['pgrep', 'ollama'], capture_output=True, text=True)
    if result.stdout.strip():
        print("Un proceso 'ollama' ya parece estar corriendo. Asumiendo que es el servidor.")
        # Intentar listar para confirmar que es el servidor y está respondiendo
        try:
            ollama.list()
            print("Ollama server está respondiendo.")
        except Exception:
            print("Proceso 'ollama' encontrado, pero no responde como servidor. Intentando iniciar uno nuevo.")
            # Si pgrep encuentra algo pero no es el servidor, podríamos intentar matarlo y reiniciar,
            # pero eso es más complejo y riesgoso. Por ahora, intentaremos iniciar uno.
            # Si el puerto ya está en uso, Popen fallará o el nuevo no iniciará correctamente.
            ollama_process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print("Intentando iniciar un nuevo Ollama server en segundo plano...")
            time.sleep(5) # Darle tiempo para que inicie
    else:
        # Si no hay proceso ollama corriendo, iniciar uno.
        ollama_process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("Intentando iniciar Ollama server en segundo plano...")
        time.sleep(5) # Darle tiempo para que inicie

except FileNotFoundError:
    print("Error: El comando 'ollama' no se encontró. Asegúrate que la instalación en la celda anterior fue correcta y está en el PATH.")
    # No continuar si ollama no se encuentra
    raise RuntimeError("Ollama no encontrado. Verificar instalación.")
except Exception as e:
    print(f"Excepción al verificar/intentar iniciar ollama serve: {e}")
    # Continuar de todas formas para intentar conectar, podría haber sido un problema temporal con pgrep.

# Esperar un poco más y verificar conexión con el servidor
print("Esperando a que el servidor Ollama esté completamente listo (10-15 segundos más)...")
time.sleep(15) # Aumentado el tiempo de espera por si acaso

try:
    ollama.list() # Verifica la conexión y lista modelos si alguno está descargado
    print("--- Conexión con Ollama server establecida. ---")
except Exception as e:
    print(f"--- No se pudo conectar a Ollama server: {e} ---")
    print("--- Por favor, asegúrate de que 'ollama serve' esté ejecutándose correctamente. ---")
    print("--- Puedes intentar ejecutar '!ollama serve &' en una celda separada o verificar los logs de Ollama. ---")
    # Considerar levantar una excepción si la conexión es crucial para los siguientes pasos
    # raise RuntimeError("Fallo al conectar con Ollama server.")

# Función para descargar modelos de Ollama (reutilizada)
def pull_ollama_model(model_name: str):
    """Descarga un modelo de Ollama si no está ya presente."""
    print(f"\nVerificando/Descargando modelo Ollama: {model_name}...")
    try:
        models_info = ollama.list()
        if 'models' in models_info:
            model_found = any(m['name'].startswith(model_name) for m in models_info['models'])
            if model_found:
                print(f"Modelo '{model_name}' ya disponible localmente.")
                return True
        else: # Si 'models' no está en la respuesta, algo raro pasa
            print(f"Advertencia: La respuesta de ollama.list() no contiene la clave 'models'. {models_info}")
    except Exception as e:
        print(f"No se pudo listar modelos de Ollama (el servidor podría no estar completamente listo o accesible): {e}")
        print("Se intentará descargar el modelo de todas formas.")

    print(f"Descargando modelo '{model_name}'... esto puede tardar.")
    try:
        current_digest = ""
        status_line = ""
        for progress in ollama.pull(model_name, stream=True):
            if 'digest' in progress:
                current_digest = progress['digest']

            # Construir una línea de estado más informativa
            status_parts = []
            if progress.get('status'):
                status_parts.append(progress.get('status'))
            if progress.get('total') and progress.get('completed'):
                completed_gb = progress.get('completed', 0) / (1024**3)
                total_gb = progress.get('total', 0) / (1024**3)
                if total_gb > 0 :
                    percentage = (progress.get('completed', 0) / progress.get('total', 0)) * 100
                    status_parts.append(f"{completed_gb:.2f}/{total_gb:.2f} GB ({percentage:.1f}%)")
                else:
                    status_parts.append(f"{completed_gb:.2f} GB descargados")


            new_status_line = " ".join(status_parts)
            if new_status_line != status_line: # Solo imprimir si cambia para evitar spam
                status_line = new_status_line
                print(f"\rDescargando {model_name}: {status_line}", end="")

            if progress.get('status') == 'success':
                print(f"\nModelo '{model_name}' descargado/verificado correctamente.")
                return True
        # Si el stream termina sin un 'success' explícito pero sin errores, asumimos que está bien.
        print(f"\nProceso de descarga/verificación de '{model_name}' completado.")
        return True

    except Exception as e:
        print(f"\n--- Error descargando el modelo '{model_name}': {e} ---")
        return False

# Nombres de los modelos LLM a utilizar (ajusta según tus necesidades y disponibilidad)
# Es buena idea usar modelos más pequeños para pruebas iniciales en Colab si los recursos son limitados.
OLLAMA_MODEL_DEEPSEEK = 'deepseek-r1:1.5b'
OLLAMA_MODEL_GEMMA = 'gemma3:4b'
OLLAMA_QWEN = 'qwen3:1.7b'
# Descargar los modelos LLM definidos
print("\n--- Iniciando descarga/verificación de modelos LLM ---")
modelos_a_descargar = [OLLAMA_MODEL_DEEPSEEK, OLLAMA_MODEL_GEMMA, OLLAMA_QWEN]
modelos_descargados_ok = {}

for model_tag in modelos_a_descargar:
    if model_tag: # Asegurarse que la variable no es None o vacía
        success = pull_ollama_model(model_tag)
        modelos_descargados_ok[model_tag] = success
    else:
        print(f"Advertencia: Una de las variables de modelo está vacía, se omite la descarga.")

print("\n--- Resumen de descarga de modelos ---")
for model_tag, status in modelos_descargados_ok.items():
    print(f"Modelo {model_tag}: {'Descargado/Verificado OK' if status else 'FALLÓ LA DESCARGA/VERIFICACIÓN'}")


print("\n--- Configuración de Ollama y descarga de modelos completada (o intentada). ---")

In [ ]:
# Celda 17 (Modificada): Configuración de ChromaDB para RAG con MiniLM
# -------------------------------------------------------------------
# Prepara la base de datos vectorial ChromaDB con los embeddings de 'paraphrase-multilingual-MiniLM-L12-v2'.
# La colección se re-creará cada vez que se ejecute esta celda.

import chromadb # Asegurarse que chromadb está importado

print("\n--- Configurando ChromaDB para RAG con Embeddings MiniLM ---")

# Nombre de la colección y cliente ChromaDB
COLLECTION_NAME_RAG_MINILM = "news_rag_minilm" # Nuevo nombre para la colección
chroma_client = chromadb.Client()

# Variables globales que esta celda establecerá para la Celda 18
active_rag_collection = None
data_available_for_rag = False
active_rag_embedding_model = None # Se establecerá con la instancia de MiniLM

# Verificar si los componentes necesarios están disponibles
# active_st_model_instance (de Celda 13)
# X_st_embeddings_normalized (de Celda 13)
# df_noticias y 'clean_body' (de Celda 6)

minilm_model_loaded_celda13 = 'active_st_model_instance' in globals() and globals()['active_st_model_instance'] is not None
minilm_embeddings_available_celda13 = 'X_st_embeddings_normalized' in globals() and globals()['X_st_embeddings_normalized'].size > 0
dataframe_ready_celda6 = (
    'df_noticias' in globals() and not globals()['df_noticias'].empty and
    'clean_body' in globals()['df_noticias'].columns and
    not globals()['df_noticias']['clean_body'].str.strip().eq('').all()
)

if minilm_model_loaded_celda13 and minilm_embeddings_available_celda13 and dataframe_ready_celda6:
    current_minilm_model = globals()['active_st_model_instance']
    news_embeddings_for_rag = globals()['X_st_embeddings_normalized']

    df_noticias_ref = globals()['df_noticias']
    news_texts_for_rag = df_noticias_ref['clean_body'].tolist()

    num_docs_to_index = min(len(news_texts_for_rag), news_embeddings_for_rag.shape[0])

    if num_docs_to_index == 0:
        print("No hay documentos o embeddings (MiniLM) para indexar.")
    else:
        news_texts_for_rag = news_texts_for_rag[:num_docs_to_index]
        news_embeddings_for_rag_final = news_embeddings_for_rag[:num_docs_to_index, :] # Renombrar para claridad
        news_ids_for_rag = [f"news_minilm_{i}" for i in range(num_docs_to_index)]

        print(f"Se utilizarán {num_docs_to_index} noticias/embeddings MiniLM para indexar en ChromaDB.")

        print(f"Intentando re-crear la colección ChromaDB: '{COLLECTION_NAME_RAG_MINILM}'...")
        try:
            if COLLECTION_NAME_RAG_MINILM in [c.name for c in chroma_client.list_collections()]:
                chroma_client.delete_collection(name=COLLECTION_NAME_RAG_MINILM)
                print(f"Colección '{COLLECTION_NAME_RAG_MINILM}' existente eliminada.")

            active_rag_collection = chroma_client.create_collection(
                name=COLLECTION_NAME_RAG_MINILM,
                metadata={"hnsw:space": "cosine"} # Coseno es bueno para MiniLM normalizado
            )
            print(f"Colección '{COLLECTION_NAME_RAG_MINILM}' creada/re-creada exitosamente.")
        except Exception as e:
            print(f"Error al crear/re-crear la colección ChromaDB '{COLLECTION_NAME_RAG_MINILM}': {e}")
            active_rag_collection = None

        if active_rag_collection:
            print("Añadiendo documentos a ChromaDB...")
            try:
                active_rag_collection.add(
                    embeddings=news_embeddings_for_rag_final.tolist(),
                    documents=news_texts_for_rag,
                    ids=news_ids_for_rag
                )
                print(f"--- Datos indexados en ChromaDB. Colección '{COLLECTION_NAME_RAG_MINILM}' lista con {active_rag_collection.count()} docs. ---")
                data_available_for_rag = True
                active_rag_embedding_model = current_minilm_model # Guardar referencia al modelo MiniLM
            except Exception as e:
                print(f"Error al añadir documentos a ChromaDB: {e}")
                data_available_for_rag = False
                try:
                    chroma_client.delete_collection(name=COLLECTION_NAME_RAG_MINILM)
                except: pass
        else:
            print(f"--- No se pudo utilizar la colección ChromaDB '{COLLECTION_NAME_RAG_MINILM}'. RAG no disponible. ---")
else:
    print("--- ADVERTENCIA: No se pueden preparar datos para RAG con MiniLM. ---")
    # ... (mensajes de error detallados) ...
    data_available_for_rag = False

# Las variables globales 'active_rag_embedding_model' y 'active_rag_collection'
# ahora deberían estar listas para que la Celda 18 las use.
if not data_available_for_rag: # Si falló, asegurarse que las globales estén None
    active_rag_embedding_model = None
    active_rag_collection = None
    print("\n--- RAG con MiniLM no se configuró. Variables 'active_rag_...' pueden ser None. ---")
else:
    print(f"\n--- RAG configurado con: Modelo='{type(active_rag_embedding_model)}', Colección='{active_rag_collection.name if active_rag_collection else 'N/A'}' ---")

In [ ]:
!ollama list

In [ ]:
# Celda 18 (Modificada): Prueba de Modelos LLM con RAG (usando MiniLM para RAG)
# ---------------------------------------------------------------------------
import ollama
import traceback

# --- 1. FUNCIÓN RAG INTERNA (AJUSTADA PARA SENTENCETRANSFORMER COMO DEFAULT) ---
def _internal_ask_llm_with_rag(query: str, ollama_model_name: str, embedding_model_for_query, db_collection, k_results: int = 3) -> tuple[str, list[str]]:
    print(f"\n--- (Interno DEBUG) Iniciando _internal_ask_llm_with_rag para: '{query}' con {ollama_model_name} ---")
    context_text = "Contexto no disponible o RAG no configurado."
    retrieved_docs_list = []

    rag_componentes_listos = (
        embedding_model_for_query is not None and
        db_collection is not None and
        globals().get('data_available_for_rag', False)
    )
    # ... (prints de debug iniciales) ...
    if embedding_model_for_query:
        print(f"(Interno DEBUG) Tipo de embedding_model_for_query: {type(embedding_model_for_query)}") # Debería ser SentenceTransformer
    # ...

    if rag_componentes_listos:
        # Para MiniLM (y muchos SentenceTransformers), no se necesita prefijo de consulta.
        # Si usaras E5, aquí agregarías "query: ".
        query_for_embedding = query
        print(f"(Interno DEBUG) query_for_embedding: '{query_for_embedding}'")

        query_embedding = None
        try:
            print("(Interno DEBUG) Intentando generar embedding para la consulta...")

            # --- LÓGICA DE CODIFICACIÓN ---
            if hasattr(embedding_model_for_query, 'encode') and callable(getattr(embedding_model_for_query, 'encode')):
                # Esta es la rama principal para SentenceTransformer (como MiniLM)
                print(f"(Interno DEBUG) Usando método .encode() de {type(embedding_model_for_query)}...")
                query_embedding_raw = embedding_model_for_query.encode([query_for_embedding]) # Devuelve ndarray directamente

                if isinstance(query_embedding_raw, np.ndarray) and query_embedding_raw.ndim == 2 and query_embedding_raw.shape[0] == 1:
                    query_embedding = query_embedding_raw[0] # Obtener el vector 1D
                    print(f"(Interno DEBUG) Embedding de consulta generado. Shape: {query_embedding.shape}, dtype: {query_embedding.dtype}, Primeros 3: {query_embedding[:3]}")
                elif isinstance(query_embedding_raw, list) and len(query_embedding_raw) > 0 and isinstance(query_embedding_raw[0], np.ndarray): # Por si devuelve lista de arrays
                    query_embedding = query_embedding_raw[0]
                    print(f"(Interno DEBUG) Embedding de consulta (de lista) gen. Shape: {query_embedding.shape}, Primeros 3: {query_embedding[:3]}")
                else:
                    print(f"(Interno DEBUG) ERROR: Salida de .encode() no es el formato esperado (ndarray 2D de 1 fila). Tipo: {type(query_embedding_raw)}, Shape: {getattr(query_embedding_raw, 'shape', 'N/A')}")
                    raise ValueError("Salida inesperada del modelo de embedding.")

            elif 'FlagEmbedding' in str(type(embedding_model_for_query)): # Si por alguna razón se pasa BGE
                print("(Interno DEBUG) Usando lógica de codificación para FlagEmbedding (BGE)...")
                encoded_output = embedding_model_for_query.encode([query_for_embedding], return_dense=True, return_sparse=False, return_colbert_vecs=False)
                # ... (lógica para BGE como antes) ...
                if isinstance(encoded_output, dict) and 'dense_vecs' in encoded_output:
                    query_embedding_list = encoded_output['dense_vecs']
                    if query_embedding_list and len(query_embedding_list) > 0:
                        query_embedding = query_embedding_list[0]
                    else: raise ValueError("'dense_vecs' vacío.")
                else: raise ValueError("Salida inesperada BGE.")
            else:
                print("(Interno DEBUG) ERROR: embedding_model_for_query no es válido.")
                raise ValueError("Modelo de embedding para consulta no es válido.")

        except Exception as e_embed:
            print(f"(Interno DEBUG) EXCEPCIÓN al generar embedding de consulta: {e_embed}")
            traceback.print_exc()
            llm_answer = f"Error crítico al generar embedding para la consulta: {e_embed}"
            return llm_answer, retrieved_docs_list

        if query_embedding is not None:
            try:
                # ... (consulta a ChromaDB como antes) ...
                print(f"(Interno DEBUG) Intentando consultar ChromaDB con embedding de shape: {query_embedding.shape}...")
                results = db_collection.query(
                    query_embeddings=[query_embedding.tolist()],
                    n_results=k_results,
                    include=['documents', 'distances', 'metadatas']
                )
                # ... (procesamiento de resultados como antes) ...
                if results and results.get('documents') and results.get('documents')[0]:
                    retrieved_docs_list = results['documents'][0]
                    context_text = "\n\n---\n\n".join(retrieved_docs_list)
                # ...
            except Exception as e_query_db:
                print(f"(Interno DEBUG) EXCEPCIÓN al consultar ChromaDB: {e_query_db}")
                traceback.print_exc()
        # ...
    # ... (resto de la función: prompt, llamada a ollama.chat) ...
    # El prompt y la llamada a ollama.chat no cambian
    prompt = f"""Utiliza únicamente la siguiente información de contexto para responder la pregunta.
Si la información no está en el contexto, indica explícitamente que no puedes responder con la información proporcionada.
No inventes respuestas. Sé conciso.

Contexto Recuperado (resumen):
---
{context_text[:500]}... (y posiblemente más)
---

Pregunta: {query}

Respuesta:"""

    try:
        # ... (llamada a ollama.chat como antes) ...
        response_ollama = ollama.chat(
            model=ollama_model_name,
            messages=[{'role': 'user', 'content': prompt}]
        )
        llm_answer = response_ollama['message']['content']
    except Exception as e_llm:
        # ... (manejo de error LLM como antes) ...
        print(f"(Interno DEBUG) EXCEPCIÓN al comunicarse con Ollama ({ollama_model_name}): {e_llm}")
        traceback.print_exc()
        llm_answer = f"Error: No se pudo obtener respuesta del modelo LLM ({ollama_model_name}). Detalle: {e_llm}"

    return llm_answer, retrieved_docs_list


# --- 2. FUNCIÓN PRINCIPAL REUTILIZABLE (obtener_respuesta_llm) ---
# ESTA FUNCIÓN NO NECESITA CAMBIOS SIGNIFICATIVOS, YA QUE USA LAS VARIABLES GLOBALES
# 'active_rag_embedding_model' y 'active_rag_collection' que la Celda 17 ahora establece.
def obtener_respuesta_llm(nombre_modelo_ollama: str, pregunta_usuario: str, k_documentos_rag: int = 3, mostrar_docs: bool = True):
    print(f"\n>>> Solicitando respuesta del modelo: [{nombre_modelo_ollama}]")
    print(f"    Pregunta: '{pregunta_usuario}'")

    embedding_model = globals().get('active_rag_embedding_model')
    db_collection_rag = globals().get('active_rag_collection')

    if embedding_model:
        print(f"    (Debug RAG en obtener_respuesta_llm) Usando modelo embedding: {type(embedding_model)}")
    else:
        print("    (Debug RAG en obtener_respuesta_llm) ADVERTENCIA: active_rag_embedding_model NO encontrado.")
    if db_collection_rag:
        print(f"    (Debug RAG en obtener_respuesta_llm) Usando colección DB: {db_collection_rag.name}")
    else:
        print("    (Debug RAG en obtener_respuesta_llm) ADVERTENCIA: active_rag_collection NO encontrada.")

    # ... (validaciones de nombre_modelo_ollama y pregunta_usuario como antes) ...
    if not nombre_modelo_ollama or not isinstance(nombre_modelo_ollama, str):
        error_msg = "Error: 'nombre_modelo_ollama' debe ser un string no vacío."
        print(error_msg)
        return error_msg
    if not pregunta_usuario or not isinstance(pregunta_usuario, str):
        error_msg = "Error: 'pregunta_usuario' debe ser un string no vacío."
        print(error_msg)
        return error_msg

    respuesta_llm, documentos_recuperados = _internal_ask_llm_with_rag(
        query=pregunta_usuario,
        ollama_model_name=nombre_modelo_ollama,
        embedding_model_for_query=embedding_model,
        db_collection=db_collection_rag,
        k_results=k_documentos_rag
    )

    # ... (código para mostrar documentos y la respuesta del LLM como antes) ...
    if mostrar_docs and documentos_recuperados:
        print("\n--- Documentos Recuperados por RAG: ---")
        for i, doc in enumerate(documentos_recuperados):
            print(f"Documento RAG [{i+1}/{len(documentos_recuperados)}]:")
            print(f"{doc[:300]}...")
            print("-" * 20)
    elif mostrar_docs:
        print("\n--- No se recuperaron documentos por RAG (o RAG no está activo o no encontró resultados). ---")

    print(f"\n<<< Respuesta de [{nombre_modelo_ollama}]:\n{respuesta_llm}")
    print("-" * 70)
    return respuesta_llm

# --- 3. EJEMPLOS DE USO DE LA FUNCIÓN PRINCIPAL ---
if __name__ == '__main__':
    print("\n--- Iniciando Pruebas de Modelos LLM (usando MiniLM para RAG) ---")

    PREGUNTA_CALOR = "¿Que se sabe sobre la visita de Marco Rubio?"

    # Usar el modelo LLM más pequeño que tienes.
    # De tu lista: 'deepseek-r1:1.5b' y 'qwen3:1.7b' son los más pequeños.
    # Voy a usar 'deepseek-r1:1.5b' como ejemplo.
    MODELO_LLM_PEQUENO = 'gemma3:4b'
    # O podrías usar MODELO_LLM_PEQUENO = 'qwen3:1.7b'

    print(f"Probando con LLM pequeño: {MODELO_LLM_PEQUENO} y embeddings MiniLM para RAG.")
    obtener_respuesta_llm(nombre_modelo_ollama=MODELO_LLM_PEQUENO,
                          pregunta_usuario=PREGUNTA_CALOR,
                          k_documentos_rag=5,
                          mostrar_docs=True)

    # Puedes probar otro LLM también si quieres comparar
    MODELO_LLM_OTRO = 'qwen3:1.7b'
    print(f"\nComparando con LLM: {MODELO_LLM_OTRO} y embeddings MiniLM para RAG.")
    obtener_respuesta_llm(nombre_modelo_ollama=MODELO_LLM_OTRO,
                          pregunta_usuario=PREGUNTA_CALOR,
                          k_documentos_rag=5,
                          mostrar_docs=True)

    print("\n--- Todas las pruebas de la celda principal completadas. ---")

In [ ]:
!ollama list